In [1]:
from __future__ import annotations
from typing import Callable

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from transformers import AutoTokenizer

from helper import *

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


# Intro

# Load Data

In [3]:
# Reduced data to make it manageable for smaller systems
DATA_FILE: str = '../data/shakespeare_small.txt'

In [4]:
with open(DATA_FILE, 'r') as data_file:
    raw_text = data_file.read()

In [5]:
print(f'Number of characters: {len(raw_text):,}')

Number of characters: 50,085


# Character-Based Generation

## Encode Text into Integer Tokens

### Normalization

> - Skip?
> - lowercase?

In [6]:
def normalize_text(text: str) -> str:
    # Can be multiple actions to normalize text
    normalized_text = text.lower()
    return normalized_text

In [7]:
# TEST: Is your text normalized the way you expected?
# normalized_text = normalize_text(raw_text)
# print(normalized_text)

### Pretokenization

In [8]:
def pretokenize_text(text: str) -> list[str]:
    smaller_pieces = [char for char in text]
    return smaller_pieces

In [9]:
# TEST: Is your (normalized) text pretokenized the way you expected?
# pretokenized_text = pretokenize_text(normalized_text)
# print(pretokenized_text)

### Tokenize

In [10]:
# Combine normalization and pretokenization steps
def tokenize_text(text: str) -> list[str]:
    normalized_text: str = normalize_text(raw_text)
    pretokenized_text: list[str] = pretokenize_text(normalized_text)
    # Characters are already tokens so pretokenized text is already tokenized
    tokenized_text = pretokenized_text
    return tokenized_text

In [11]:
tokenized_text = tokenize_text(raw_text)

### Postprocessing

> Skip

### Encode (Tokens → Integer IDs)

In [12]:
# to get enocding (IDs) of text
def encode_text(
    text: str,
    tokenize_func: Callable[[str], list[str]],
) -> tuple[list[int], TokenMapping]:
    tokenized_text: list[str] = tokenize_func(text)
    # Get object for translating tokens to IDs and back
    token_mapping = TokenMapping(tokenized_text)
    # Get your text encoded as list of IDs
    enocded_text: list[str] = token_mapping.encode(tokenized_text)
    return enocded_text, token_mapping

In [13]:
encoded, character_mapping = encode_text(raw_text, tokenize_text)

## Prepare Dataset

In [14]:
n_tokens = character_mapping.n_tokens
dataset_size = len(encoded)

In [15]:
length = 32 # Number of characters
batch_size = 32

train_dataset = ShakespeareDataset(encoded, length)
train_loader = DataLoader(
    train_dataset,
    shuffle=True,
    batch_size=batch_size,
)

## Define Model

In [16]:
model = build_model(n_tokens)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

## Define Text Genaration

In [17]:
def generate_text(
    model,
    input_str: str,
    num_chars: int = 100,
    temperature: float = 1.0,
) -> str:
    # Set model into "evaluation mode" (deactivates things like Dropout layers)
    model.eval()
    generated_text = input_str.lower()
    input_tensor = tokens_to_id_tensor(
        tokens=input_str.lower(),
        token_id_mapping=character_mapping.token2id,
    )
    
    with torch.no_grad():
        for _ in range(num_chars):
            output = model(input_tensor.to(device))
            probabilities = nn.functional.softmax(
                output[0, -1] / temperature,
                dim=0,
            )
            next_char_idx = torch.multinomial(probabilities, 1).item()
            generated_text += character_mapping.id2token(next_char_idx)
            input_tensor = torch.cat(
                [
                    input_tensor,
                    torch.tensor([[next_char_idx]], dtype=torch.long),
                ],
                1,
            )
            
    return generated_text

## Train Model

In [18]:
PHRASE = 'To be or not to be'
epochs = 5

start = start_time()
for epoch in range(epochs):
    # Set model into "training mode"
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch.to(device))
        loss = criterion(output.transpose(1, 2), y_batch.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f'Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader)}')
    print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / epochs * 100, loss))
    print('-'*72)
    gen_output = generate_text(model, PHRASE, 100)
    print(gen_output)

Epoch 1/5, Loss: 2.272809115461648
 6.3s (0 0%) 2.1010]
------------------------------------------------------------------------
to be or not to be kanfiteinon we and irs olek i cood! wreiceld simy! homond boti! haruifling sy ap'l,
yhou prard digh
Epoch 2/5, Loss: 1.904132768902154
 12.6s (1 20%) 1.8678]
------------------------------------------------------------------------
to be or not to be mairadius, the chand tohtia that ag'd the cone,
secials marcius, and sout arst hary, it arthion and
Epoch 3/5, Loss: 1.818787350136632
 19.0s (2 40%) 1.7272]
------------------------------------------------------------------------
to be or not to be the,
the comand to reso done's for one o' hy you well known with itch, poo, i vereed.

coot the sho
Epoch 4/5, Loss: 1.7758593479284464
 25.4s (3 60%) 1.5742]
------------------------------------------------------------------------
to be or not to be fram! you
he prear gack ange?

mane yes feer?
'
all would shum, agrainefides thing one thanath marc
E

## Generate Text

In [19]:
output = generate_text(model, 'To be or not to be', 100)
print(output)

to be or not to bear hem? se'd beat what a:
at the vood.

brun:
i for ary the birs, would to knowe; has you have fith 


# Token-Based Generation

## Encode Text into Integer Tokens

### Tokenize (Choose)

In [20]:
# Choose a pretrained tokenizer to use
xlmr_model_name = 'xlm-roberta-base'
bert_model_name = 'bert-base-cased'
bert_model_name_uncased = 'bert-base-uncased'

my_tokenizer = AutoTokenizer.from_pretrained(
    bert_model_name_uncased,
)

### Encode (Tokens → Integer IDs)

In [21]:
# Get enocding (IDs) of text
def encode_text_from_tokenizer(
    text: str,
    tokenizer,
) -> tuple[list[int], TokenMapping]:
    # Using tokenizer, get text as list of tokens (strings)
    tokenized_text: list[str] = tokenize_text_from_tokenizer(
        tokenizer=tokenizer,
        text=text,
    )
    # Get object for translating tokens to IDs and back
    token_mapping = TokenMapping(tokenized_text)
    # Get your text encoded as list of IDs
    enocded_text: list[str] = token_mapping.encode(tokenized_text)
    return enocded_text, token_mapping

In [22]:
encoded, token_mapping = encode_text_from_tokenizer(
    text=raw_text,
    tokenizer=my_tokenizer,
)

## Prepare Dataset

In [23]:
n_tokens = token_mapping.n_tokens
dataset_size = len(encoded)

In [24]:
length = 16  # Tokens 
batch_size = 32

train_dataset = ShakespeareDataset(encoded, length)
train_loader = DataLoader(
    train_dataset,
    shuffle=True,
    batch_size=batch_size,
)

## Define Model

In [25]:
model = build_model(n_tokens)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

## Define Text Genaration

In [26]:
def generate_text(
    tokenizer,
    model,
    input_str: str,
    num_tokens: int = 100,
    temperature: float = 1.0,
) -> str:
    # Set model into "evaluation mode" (deactivates things like Dropout layers)
    model.eval()
    tokenized_text = tokenize_text_from_tokenizer(
        tokenizer=tokenizer,
        text=input_str,
    )
    input_tensor = tokens_to_id_tensor(
        tokens=tokenized_text,
        token_id_mapping=token_mapping.token2id,
    )

    generated_text = []
    with torch.no_grad():
        for _ in range(num_tokens):
            output = model(input_tensor.to(device))
            probabilities = nn.functional.softmax(
                output[0, -1] / temperature,
                dim=0,
            )
            next_token_idx = torch.multinomial(probabilities, 1).item()
            generated_text.append(token_mapping.id2token(next_token_idx))
            input_tensor = torch.cat(
                [
                    input_tensor,
                    torch.tensor([[next_token_idx]], dtype=torch.long),
                ],
                1,
            )
    # Convert to text again
    output_ids = tokenizer.convert_tokens_to_ids(generated_text)
    output_str = input_str + ' ' + tokenizer.decode(output_ids)
    return output_str

## Train Model

In [27]:
epochs = 5

start = start_time()
for epoch in range(epochs):
    # Set model into "training mode"
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch.to(device))
        loss = criterion(output.transpose(1, 2), y_batch.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f'Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader)}')
    print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / epochs * 100, loss))
    print('-'*72)
    output = generate_text(my_tokenizer, model, 'To be or not to be', 30)
    print(output)


Epoch 1/5, Loss: 6.206475119811195
 2.7s (0 0%) 5.8849]
------------------------------------------------------------------------
To be or not to be , here believe bad quarry auf! my country nor base for,! strike would'had?, cometinothini rushes thena myxtli
Epoch 2/5, Loss: 5.448414956856238
 5.5s (1 20%) 5.4670]
------------------------------------------------------------------------
To be or not to be tis below with make can wine ladiesria, to say will, come, upon wine's nape? no i, you : not again convenient ;
Epoch 3/5, Loss: 4.9026114018294065
 8.3s (2 40%) 5.1810]
------------------------------------------------------------------------
To be or not to be count.a and, advanced, and two upon i think wars fabric to co love in backs peace : the more, itcal thousandly almost free
Epoch 4/5, Loss: 4.49007889824192
 11.1s (3 60%) 4.8475]
------------------------------------------------------------------------
To be or not to be go are bo in these easy dispatch sure can their battle in t

## Generate Text

In [28]:
output = generate_text(my_tokenizer, model, 'To be or not to be', 30)
print(output)

To be or not to be ##see you him slip aim! mygeny, to all whereinst why'er good general think. second citizen : soft thou a nature, we me


# Comparison Between Generation